In [ ]:
!pip install remotezip 
!pip install tqdm 
!pip install opencv-python==4.5.2.52
!pip install opencv-python-headless==4.5.2.52 
!pip install tf-models-official

In [1]:
import tqdm
import random
import pathlib
import itertools
import collections

import cv2
import numpy as np
import remotezip as rz
import seaborn as sns
import matplotlib.pyplot as plt

import keras
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Import the MoViNet model from TensorFlow Models (tf-models-official) for the MoViNet model
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model

2024-09-04 16:26:06.070276: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-04 16:26:06.080104: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-04 16:26:06.090519: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-04 16:26:06.093549: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-04 16:26:06.101478: I tensorflow/core/platform/cpu_feature_guar

In [2]:
def split_class_lists(files_for_class, count):
  split_files = []
  remainder = {}
  for cls in files_for_class:
    split_files.extend(files_for_class[cls][:count])
    remainder[cls] = files_for_class[cls][count:]
  return split_files, remainder

def format_frames(frame, output_size):
  frame = tf.image.convert_image_dtype(frame, tf.float32)
  frame = tf.image.resize_with_pad(frame, *output_size)
  return frame

def frames_from_video_file(video_path, n_frames, output_size = (224,224), frame_step = 15):
  # Read each video frame by frame
  result = []
  src = cv2.VideoCapture(str(video_path))  

  video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

  need_length = 1 + (n_frames - 1) * frame_step

  if need_length > video_length:
    start = 0
  else:
    max_start = video_length - need_length
    start = random.randint(0, max_start + 1)

  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  # ret is a boolean indicating whether read was successful, frame is the image itself
  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  return result

class FrameGenerator:
  def __init__(self, path, n_frames, training = False):
    self.path = path
    self.n_frames = n_frames
    self.training = training
    self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir()))
    self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

  def get_files_and_class_names(self):
    video_paths = list(self.path.glob('*/*.avi'))
    classes = [p.parent.name for p in video_paths] 
    return video_paths, classes

  def __call__(self):
    video_paths, classes = self.get_files_and_class_names()

    pairs = list(zip(video_paths, classes))

    if self.training:
      random.shuffle(pairs)

    for path, name in pairs:
      video_frames = frames_from_video_file(path, self.n_frames) 
      label = self.class_ids_for_name[name] # Encode labels
      yield video_frames, label

Create the training and test datasets:

In [8]:
from pathlib import Path
batch_size = 8
num_frames = 8

# Define the subset paths based on your downloaded dataset
subset_paths = {
    'train': Path('dataset/train'),
    'test': Path('dataset/test')
}

output_signature = (
    tf.TensorSpec(shape=(None, None, None, 3), dtype=tf.float32),
    tf.TensorSpec(shape=(), dtype=tf.int16)
)

train_ds = tf.data.Dataset.from_generator(
    FrameGenerator(subset_paths['train'], num_frames, training=True),
    output_signature=output_signature
).batch(batch_size)

test_ds = tf.data.Dataset.from_generator(
    FrameGenerator(subset_paths['test'], num_frames),
    output_signature=output_signature
).batch(batch_size)


I0000 00:00:1725447708.527382   36630 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-04 16:31:48.553642: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
for frames, labels in train_ds.take(10):
  print(labels)

tf.Tensor([0 4 1 4 4 4 3 0], shape=(8,), dtype=int16)
tf.Tensor([1 4 0 1 0 0 0 0], shape=(8,), dtype=int16)
tf.Tensor([4 3 2 2 1 0 0 1], shape=(8,), dtype=int16)
tf.Tensor([0 0 1 4 2 0 0 0], shape=(8,), dtype=int16)
tf.Tensor([2 4 4 3 4 0 2 3], shape=(8,), dtype=int16)
tf.Tensor([2 2 0 1 2 3 4 1], shape=(8,), dtype=int16)
tf.Tensor([2 1 2 2 0 0 3 3], shape=(8,), dtype=int16)
tf.Tensor([1 3 1 1 4 4 2 4], shape=(8,), dtype=int16)
tf.Tensor([4 2 2 3 3 4 2 1], shape=(8,), dtype=int16)
tf.Tensor([1 0 1 2 0 0 3 3], shape=(8,), dtype=int16)


2024-09-04 16:31:55.266386: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Take a look at the shape of the data.

In [10]:
print(f"Shape: {frames.shape}")
print(f"Label: {labels.shape}")

Shape: (8, 8, 224, 224, 3)
Label: (8,)


In [11]:
gru = layers.GRU(units=4, return_sequences=True, return_state=True)

inputs = tf.random.normal(shape=[1, 10, 8]) # (batch, sequence, channels)

result, state = gru(inputs) # Run it all at once

In [12]:
first_half, state = gru(inputs[:, :5, :])   # run the first half, and capture the state
second_half, _ = gru(inputs[:,5:, :], initial_state=state)  # Use the state to continue where you left off.

print(np.allclose(result[:, :5,:], first_half))
print(np.allclose(result[:, 5:,:], second_half))

True
True


In [13]:
model_id = 'a0'
resolution = 224

tf.keras.backend.clear_session()

backbone = movinet.Movinet(model_id=model_id)
backbone.trainable = False

# Set num_classes=600 to load the pre-trained weights from the original model  
model = movinet_model.MovinetClassifier(backbone=backbone, num_classes=600)
model.build([None, None, None, None, 3])

# Load pre-trained weights
!wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a0_base.tar.gz -O movinet_a0_base.tar.gz -q
!tar -xvf movinet_a0_base.tar.gz

checkpoint_dir = f'movinet_{model_id}_base'
checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint = tf.train.Checkpoint(model=model)
status = checkpoint.restore(checkpoint_path)
status.assert_existing_objects_matched()

movinet_a0_base/
movinet_a0_base/checkpoint
movinet_a0_base/ckpt-1.data-00000-of-00001
movinet_a0_base/ckpt-1.index


In [14]:
def build_classifier(batch_size, num_frames, resolution, backbone, num_classes):
  """Builds a classifier on top of a backbone model."""
  model = movinet_model.MovinetClassifier(
      backbone=backbone,
      num_classes=num_classes)
  model.build([batch_size, num_frames, resolution, resolution, 3])

  return model

In [15]:
model = build_classifier(batch_size, num_frames, resolution, backbone, 10)

In [16]:
num_epochs = 5
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(loss=loss_obj, optimizer='adam', metrics=['accuracy'])


In [17]:
results = model.fit(train_ds,
                    validation_data=test_ds,
                    epochs=num_epochs,
                    validation_freq=1,
                    verbose=1)

Epoch 1/5


I0000 00:00:1725447804.466275   37947 service.cc:146] XLA service 0x7d421802a8f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1725447804.466301   37947 service.cc:154]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1725447804.474281   37947 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


      1/Unknown - 12s 12s/step - loss: 2.2928 - accuracy: 0.0000e+00

2024-09-04 16:33:31.380706: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


     19/Unknown - 124s 6s/step - loss: 0.9708 - accuracy: 0.7600

2024-09-04 16:35:23.154618: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


19/19 [==============================] - 204s 11s/step - loss: 0.9708 - accuracy: 0.7600 - val_loss: 0.2405 - val_accuracy: 0.9700
Epoch 2/5
19/19 [==============================] - 193s 10s/step - loss: 0.2116 - accuracy: 0.9200 - val_loss: 0.1100 - val_accuracy: 0.9800
Epoch 3/5
19/19 [==============================] - 191s 10s/step - loss: 0.1009 - accuracy: 0.9467 - val_loss: 0.1267 - val_accuracy: 0.9600
Epoch 4/5
19/19 [==============================] - 195s 10s/step - loss: 0.0577 - accuracy: 0.9933 - val_loss: 0.0316 - val_accuracy: 1.0000
Epoch 5/5
19/19 [==============================] - 193s 10s/step - loss: 0.0280 - accuracy: 0.9933 - val_loss: 0.0549 - val_accuracy: 0.9900


In [18]:
model.evaluate(test_ds, return_dict=True)

13/13 [==============================] - 78s 6s/step - loss: 0.0639 - accuracy: 0.9800


2024-09-04 16:50:58.584413: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


{'loss': 0.06390444934368134, 'accuracy': 0.9800000190734863}

In [19]:
def get_actual_predicted_labels(dataset):
  actual = [labels for _, labels in dataset.unbatch()]
  predicted = model.predict(dataset)

  actual = tf.stack(actual, axis=0)
  predicted = tf.concat(predicted, axis=0)
  predicted = tf.argmax(predicted, axis=1)

  return actual, predicted

In [20]:
def plot_confusion_matrix(actual, predicted, labels, ds_type):
  cm = tf.math.confusion_matrix(actual, predicted)
  ax = sns.heatmap(cm, annot=True, fmt='g')
  sns.set(rc={'figure.figsize':(12, 12)})
  sns.set(font_scale=1.4)
  ax.set_title('Confusion matrix of action recognition for ' + ds_type)
  ax.set_xlabel('Predicted Action')
  ax.set_ylabel('Actual Action')
  plt.xticks(rotation=90)
  plt.yticks(rotation=0)
  ax.xaxis.set_ticklabels(labels)
  ax.yaxis.set_ticklabels(labels)

In [21]:
fg = FrameGenerator(subset_paths['train'], num_frames, training = True)
label_names = list(fg.class_ids_for_name.keys())

In [22]:
actual, predicted = get_actual_predicted_labels(test_ds)
plot_confusion_matrix(actual, predicted, label_names, 'test')

13/13 [==============================] - 80s 6s/step
